<div style="color:Navy"> 

<div style="text-align:center"> 

***
# <u>TP3:</u>
# Bruit et Opérations morhpologiques
    
<p style="text-align: center; color:gray"><i>@Author:</i> Marc-Aurèle Rivière</p>

***
        
</div>
    
<u>**Plan:**</u>

1. [**Bruit et Débruitage**](#1)
2. [**Opérations morphologiques**](#2): 
    1. Erosion & Dilatations
    2. Ouverture & Fermetures
    3. Hit-or-Miss
    4. Thinning & Thickening
    5. Skeletonization
    6. Convex Hull
    
</div>

In [1]:
'''''''''''''''''''''''''''''''''
#################################
#  Code global pour tout le TP  #
#################################
'''''''''''''''''''''''''''''''''

import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("Running on Colaboratory")
    from google.colab import drive, files

    drive.mount('/content/gdrive')
    root_path = 'gdrive/My Drive/3. Doctorat/Enseignements/[Intro] Image Processing/TP3/'
    img_path = root_path + "img/"
    morpho_path = img_path + "morpho/"
else:
    print("Not running on Colaboratory")
    root_path = "/"
    img_path = "img/"
    morpho_path = img_path + "morpho/"

Running on Colaboratory
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


# <span style="color: green;text-decoration: underline" id="1">I. Bruit et Débruitage</span>
***

## <span style="color: DodgerBlue;text-decoration: underline">I.1 Bruit</span>
***

En traitement du signal (et donc par extension, des images), le **bruit** fait référence à des variations aléatoires "non-expliquées" du signal (et donc de la luminance ou couleur des pixels dans notre cas). Ce bruit est donc une dégradation de l'image par rapport à un état d'origine, ou un état optimal espéré.

**Ajouter du bruit** à une image revient donc à modifier la valeur de luminance (ou chrominance) de certains pixels par des valeurs "aléatoires". En réalité, le bruit observé dans les systèmes électroniques n'est pas aléatoire, mais dépends de processus génératifs qui ne sont pas connus / compris, et est donc considéré / modélisé comme aléatoire. Ce bruit peut  provenir de plusieurs sources: mauvais capteurs, trop faible luminosité lors de la prise de l'image, problèmes d'électronique, dégradation du support de stockage de l'image avec le temps, ...
  
On distingue deux grandes catégories de bruits:

### I.1.a Bruits additifs :

Le niveau de bruit est indépendant de la valeur du signal: chaque pixel est uniformément susceptibles d'être bruité. Il n'y à donc pas d'intéraction entre le bruit et le signal.

$I'(x,y) = I(x,y) + \eta(x,y)$

<u>Exemples de bruits additifs</u>: Gaussian, ...

Les **filtres linéaires** sont efficaces pour retirer les bruits additifis.

### I.1.b Bruits multiplicatifs :

Le niveau de bruit dépends / est corrélé à la valeur du signal: certains pixels seront plus susceptibles d'être bruités selon leur valeur. On peut parler d'interaction entre le bruit et le signal.

<u>Exemples de bruits multiplicatifs</u>: Salt, Pepper, Salt&Pepper, Speckle, Poisson, ...

Le **filtre médian** et les **opération morphologiques** sont efficaces pour retirer les bruits multiplicatifs.

In [0]:
### Code utile pour cette section :
import os

from PIL import Image
from skimage.util import random_noise
import numpy as np
import cv2

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from ipywidgets import interact, interact_manual
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

# Fonction d'affichage utilisant OpenCV pour calculer l'histogramme de l'image
def affichage_2x2(img1, img2, c="gray"):
    fig = plt.figure(figsize=(18, 12))
    
    ax1 = plt.subplot(221)
    ax1.imshow(img1, c)
    ax1.set_title("Image originale", color='b')
    ax2 = plt.subplot(222)
    ax2.hist(img1.ravel(), 256, [0,256])
    ax2.set_title("Histogramme de l'image originale", color='b')
    
    ax3 = plt.subplot(223)
    ax3.imshow(img2, c)
    ax3.set_title("Image bruitée", color='c')
    ax4 = plt.subplot(224, sharex=ax2, sharey=ax2)
    ax4.hist(img2.ravel(), 256, [0,256])
    ax4.set_title("Histogramme de l'image bruitée", color='c')
    plt.show()

# Affichons l'image originale, modifiée, et leurs histogrames (*smoothed* via un KDE - Kernel Density Estimation)
def affichage_2x2_smoothed(img1, img2):
    fig = plt.figure(figsize=(18, 12))

    ax1 = plt.subplot(221)
    ax1.imshow(img1, "gray")
    ax1.set_title("Image originale")
    ax2 = plt.subplot(222)
    sns.kdeplot(np.array(img1).ravel(), shade=True, kernel="gau", bw="scott", clip=[0,256], cut=0)
    ax2.set_title("Histogramme (KDE) de l'original")
    
    ax3 = plt.subplot(223)
    ax3.imshow(img2, "gray")
    ax3.set_title("Image bruitée")
    ax4 = plt.subplot(224, sharex=ax2, sharey=ax2)
    sns.kdeplot(np.array(img2).ravel(), shade=True, kernel="gau", bw="scott", clip=[0,256], cut=0)
    ax4.set_title("Histogramme (KDE) de l'image bruitée")
    plt.show()

### I.1.a Bruit Gaussien

Un **bruit gaussien** est un **bruit additif** distribué selon une **loi Normale**, ajouté à l'ensemble de l'image. Cela signifie que chaque pixel de l'image va se voir ajouter une valeur piochée aléatoirement dans une distribution Gaussienne :
    
$X \sim \mathcal{N}(\mu,\,\sigma^{2})$

L'intensité du bruit est lié à la variance $\sigma^2$ de la distribution utilisée pour modéliser ce bruit. Plus $\sigma^2$ augmente, plus l'histogramme de l'image bruité se rapprochera d'une loi Normale, le bruit dominant progressivement le signal.

La nouvelle intensité de chaque pixel s'exprime par :

$I'(x,y) = I(x,y) + \mathcal{N}(\mu,\,\sigma^{2})$

In [0]:
@interact
def gaussian_noise(image=[f for f in os.listdir(img_path) if os.path.isfile(os.path.join(img_path, f))], mean=(-2,2,0.5), var=(0,30,1)):
    img = np.array(Image.open(img_path + image).convert("L"))
    
    # Ajoutons un bruit Gaussien (normal) de moyenne $mean$ et variance $var$
    noisy = img + np.random.normal(mean, var, img.shape)
    noisy = np.clip(noisy, 0, 255) 

    # On affiche
    affichage_2x2(img, noisy)

interactive(children=(Dropdown(description='image', options=('lena.jpg', 'cells.png', 'barbara.jpg', 'valve.pn…

### I.1.b Bruit Salt-and-Pepper (S&P)

Un **bruit de Poisson** (ou *Impulse noise*, ou *Spike noise*) est un **bruit multiplicatif** qui va affecter un certain % de l'image. Cela signifie que chaque pixel de l'image va avoir une certaine chance de passer à 0 (Pepper) ou 255 (Salt), les pixels noirs ayant une plus grande chance de devenir blancs, et vice-versa.

*Salt corresponds to pixels in a dark region that somehow passed the threshold for bright, and pepper corresponds to pixels in a bright region that were below threshold. Salt and pepper might be classification errors resulting from variation in the surface material or illumination, or perhaps noise in the analog/digital conversion process in the frame grabber.*

In [0]:
@interact
def sp_noise(image=[f for f in os.listdir(img_path) if os.path.isfile(os.path.join(img_path, f))], sp_amount=(0,1,0.05)):
    img = np.array(Image.open(img_path + image).convert("L"))
    
    # Ajoutons un bruit Gaussien (normal) de moyenne $mean$ et variance $var$
    noisy_sp = random_noise(img, mode='s&p', amount=sp_amount)
    # The above function returns a floating-point image on the range [0, 1], thus we changed it to 'uint8'
    noisy_sp = np.array(255 * noisy_sp, dtype='uint8')

    # On affiche
    affichage_2x2(img, noisy_sp)

interactive(children=(Dropdown(description='image', options=('lena.jpg', 'cells.png', 'barbara.jpg', 'valve.pn…

### I.1.c Poisson noise:

Un **bruit de Poisson** (ou *Shot noise*) est un **bruit multiplicatif** modélisé par une **loi de Poisson**, appliqué à toute l'image. Cela signifie que chaque pixel de l'image va voir sa valeur recalculée selon la loi de Poisson, de moyenne et variance égales à $\lambda$:

$X \sim \mathcal P(\lambda)$, avec $P\left( x \right) = \dfrac{{e^{ - \lambda } \lambda ^x }}{{x!}}$

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/16/Poisson_pmf.svg/440px-Poisson_pmf.svg.png">

<u>Remarque</u>: Plus $\lambda$ est grand, et plus la distribution de Poisson se rapproche d'une loi Normale.

In [0]:
@interact
def poisson_noise(image=[f for f in os.listdir(img_path) if os.path.isfile(os.path.join(img_path, f))], lambda_const=(0,50,5)):
    img = np.array(Image.open(img_path + image).convert("L"))
    
    noisy = np.random.poisson(img / 255.0 * lambda_const) / lambda_const * 255
    noisy = np.clip(noisy, 0, 255).astype(np.uint8)

    # On affiche
    affichage_2x2(img, noisy)

interactive(children=(Dropdown(description='image', options=('lena.jpg', 'cells.png', 'barbara.jpg', 'valve.pn…

### <span style="color:crimson">**[<u>Exercice</u>]** A vous de jouer:</span>
***
<div style="color:DarkSlateBlue">  
 
1. **Créez une méthode interactive permettant de choisir l'un des trois types de bruits mentionnés et de l'appliquer à une image couleur.**
    
    
2. **Implémentez deux méthodes permettant de rajouter du bruit *Salt* et *Pepper* séparément.**

</div>

In [0]:
# > Emplacement exercice <



## <span style="color: DodgerBlue;text-decoration: underline">I.2 Débruitage</span>
***

L'objectif général des méthodes de débruitage est de retrouver au mieux l'image d'origine à partir d'une image bruitée, ce qui nécessite (1) de deviner quels pixels sont bruités, et (2) d'estimer la valeur optimale que ces pixels devraient avoir, en fonction de celles des pixels alentours. On parle **d'interpolation** des valeurs.

Estimer le type de bruit auquel le signal est soumis (en fonction des sources suspectées de bruit dans le système) va influencer la stratégie de débruitage à appliquer. De manière général, ces stratégies tentent de déterminer quelles variations d'intensité sont du bruit ou des détails réels (signal) de l'image afin de moyenner/filtrer le bruit tout en préservant les détails.

Aucun de ces algorithmes n'est parfait, et ils entraînent souvent une perte d'information dans l'image.

<img src="https://storage.googleapis.com/pagina-personal.appspot.com/img_blog/dual_rof_denoising/gradient_descent_dual_rof.png" heigth="400">

In [0]:
### Code utile pour cette section :
import os

from PIL import Image
from skimage.util import random_noise
import numpy as np
import cv2

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from ipywidgets import interact
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

# Ajout de bruit Gaussien
def gaussian_noise(img, mean = 0.0, std = 10.0):
    noisy_gauss = img + np.random.normal(mean, std, img.shape)
    return np.array(np.clip(noisy_gauss, 0, 255), dtype='uint8')

# Ajout de bruit Salt&Pepper
def sp_noise(img, prob=0.05):
    noisy_sp = random_noise(img, mode='s&p', amount=prob)
    # The above function returns a floating-point image on the range [0, 1], thus we changed it to 'uint8'
    return np.array(np.clip(noisy_sp, 0, 255), dtype='uint8')

def affichage_1x3(img1, img2, img3):
    fig = plt.figure(figsize=(18, 6))
    
    plt.subplot(131), plt.imshow(img1, cmap='gray'), plt.title("Image originale", color='b')
    plt.subplot(132), plt.imshow(img2, cmap='gray'), plt.title("Image bruitée", color='b')
    plt.subplot(133), plt.imshow(img3, cmap='gray'), plt.title("Image débruitée", color='b')
    plt.show()

#### Pour une image en noir et blanc:

**Avec OpenCV:**
```Python
imgDenoised = cv2.fastNlMeansDenoising(image, None, h, templateWindowSize, searchWindowSize)
```
Qui se base sur l'algorithme *Non-local Means Denoising*

**Avec:**
- ```h```: la puissance du filtre (une valeur autour de 10 donnera de bons résultats)
- ```templateWindowSize``` et ```searchWindowSize``` deux paramètres influençant la taille du filtre. Doivent avoir des valeurs impaires.

In [0]:
@interact
def denoising_bw(image=[f for f in os.listdir(img_path) if os.path.isfile(os.path.join(img_path, f))], 
                            var=(0,30,1), h =(7, 13, 1), templateWindowSize=(3,13,2), searchWindowSize=(5,15,2)):
    
    img = np.array(Image.open(img_path + image).convert("L"))
    
    noisy = gaussian_noise(img, 0.0, var)
    
    denoised = cv2.fastNlMeansDenoising(noisy, None, h, templateWindowSize, searchWindowSize) # 10, 9, 13

    # On affiche
    affichage_1x3(img, noisy, denoised)

interactive(children=(Dropdown(description='image', options=('barbara.jpg', 'car.png', 'cells.png', 'chest.jpg…

#### Pour une image en couleur:

**Avec OpenCV:**
```Python
imgDenoised = cv2.fastNlMeansDenoisingColored(image, None, h, hColor, templateWindowSize, searchWindowSize)
```

Le seul paramètre qui change est hColor, auquel on donne typiquement la même valeur que h.

In [0]:
@interact
def denoising_color(image=[f for f in os.listdir(img_path) if os.path.isfile(os.path.join(img_path, f))], 
                            var=(0,30,1), h =(7, 13, 1), templateWindowSize=(3,13,2), searchWindowSize=(5,15,2)):
    
    img = np.array(Image.open(img_path + image))
    
    noisy = gaussian_noise(img, 0.0, var)
    
    denoised = cv2.fastNlMeansDenoisingColored(noisy, None, h, h, templateWindowSize, searchWindowSize)

    # On affiche
    affichage_1x3(img, noisy, denoised)

interactive(children=(Dropdown(description='image', options=('lena.jpg', 'cells.png', 'barbara.jpg', 'valve.pn…

**Remarque**: débruiter une image consister à déterminer la "valeur optimale" pour les pixels bruités, afin que l'image retrouve son aspect d'origine (qui n'est pas connu de l'algorithme). Pour ce faire, l'algorithme va tenter de déterminer quels pixels sont bruités, et de modifier leur valeurs pour qu'ils soient plus "congruents" avec leurs voisins.

Cette opération est similaire à une opération de filtrage (cf. <a href="#3">partie 3</a> du TP), qui sont très souvent utilisés pour débruiter une image.

# <span style="color: green;text-decoration: underline" id="2">II. Operations morphologiques</span>
***

Les **opérations morphologiques** sont un ensemble **d'opérations locales non-linéaires** modifiant la forme (ou morphologie) du contenu d'une image. Ces opérations permettent d'altérer (faire ressortir ou effacer) certaines caractéristiques des images, comme les bordures ou les coins. La **morphologie mathématique** fait référence à une collection de méthodes de traitement d'image pour analyser et modifier des formes spatiales simples, généralement sur des images en noir et blanc.

Ces opérations se basent uniquement sur la **position relative des pixels** (comment ils sont agencés spatialement les uns par rapport aux autres), et n'est pas affecté par leurs valeurs, ce qui les rends propices au **traitement d'images binaires**.

Chaque opération morphologique petite matrice binaire (généralement imapire et de taille 3x3), appelée **élément structurant** (ES), qui **encode une caractéristique structurelle / motif 2D / prototype de forme géométrique recherché dans l'image**. Les ES spécifient donc des conditions sur l'agencement spatial des pixels que l'on cherche à matcher avec l'image, soit pour trouver une correspondance partielle (*hit*), totale (*fit*), ou un absence de correspondance (*miss*). 

Cet ES sera déplacé sur l'ensemble de l'image (*sliding window*) et combiné / matché à chaque pixel selon un certain opérateur : intersection, union, inclusion, complément. Les ES sont égalements appelés **noyaux** (*kernels*), ou encores des **masques**.

Cette combinaison consiste à vérifier que le voisinnage du pixel évalué "correspond" à l'élément structurant (inclusion, intersection, ...). Le résultat de cette combinaison changera la valeur du pixel original (1 [soit 255] si le test à été un succès (*hit*), 0 sinon (*miss*)), créant ainsi une nouvelle image binaire.

<img src="https://www.cs.auckland.ac.nz/courses/compsci773s1c/lectures/ImageProcessing-html/morph-probing.gif"> | <img src="https://drive.google.com/uc?id=1BkDWys9n8JNSUpMWM2rCNaLapALjMEep">
:--:|:--:  

Exemples d'éléments structurants:  

<img src="https://drive.google.com/uc?id=1hJ789nqs0FX0GyvvKh5rOC9u6u8Ze6ED"> | <img src="https://drive.google.com/uc?id=1-HLGgoLoyeZ9km7oHMfEDbbHLgqBPxrp" width="40%">
:--:|:--: 

***
Les opérations morphologiques peuvent être utilisées pour :
* Débruiter une image
* Améliorer le contraste d'une image
* Extraction de caractéristiques: bordures, squelette
* Modification de caractéristiques: affinnement / épaississement des bordures, ...
* Matching (detection) de caractéristiques

Il existe de nombreuses opérations morphologiques : *Erosion, Dilation, Opening, Closing, Thinning, Thickening, Top-Hat, Black-Hat, Hit & Miss, Skeletonization, ...*


***
Le type d'opération morphologique est conditionné par:
* Le type d'opérateur employé pour combiner l'image et l'ES (intersection, union, inclusion, complément).
* Le type d'ES utilisé.
* Comment ces différents opérateurs et ES sont enchaînés (quand plusieurs sont employés successivement).

La plupart de ces opérations sont une combinaison de deux opérations de base: érosion et dilatation.

In [0]:
### Code utile pour cette section :
import os

from PIL import Image, ImageOps
from skimage.util import random_noise
import numpy as np
import cv2

import seaborn as sns
from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt
%matplotlib inline

import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import display, Markdown

def affichage_1x3(img1, img2, img3, grid=True):
    fig = plt.figure(figsize=(18, 6))
    gs  = GridSpec(1, 3, width_ratios=[1, 0.4, 1])
    display_axes = "both" if grid else "none"
    
    plt.subplot(gs[0]), plt.imshow(img1, cmap='gray'), plt.title("Image", color='b')
    
    plt.subplot(gs[1]), plt.imshow(img2, cmap='gray', extent=(0, img2.shape[0], img2.shape[1], 0), origin='upper')
    plt.title("ES"), plt.locator_params(nbins=len(img2)), plt.grid(axis=display_axes, color='b', linewidth=1)
    
    plt.subplot(gs[2]), plt.imshow(img3, cmap='gray'), plt.title("Resultat", color='b')
    plt.show()

## <span style="color: DodgerBlue;text-decoration: underline">II.1 Erosion & Dilatation</span>

### II.1.a Erosion:

**L'érosion** (*Erode / Shrink / Reduce*) est une des opérations morphologiques de base, qui **érode les bordures** des objets / régions saillantes (*foreground regions*), i.e. là où les pixels sont blancs (valent 1).

Cas d'utilisation de l'érosion :
* Eliminer du bruit blanc (*Salt*) du fond (*background* - noir) de l'image.
* Affiner / amaincir les formes d'intérêt (*foreground* - blanc).

Mathématiquement, l'érosion est formalisée comme : $I' = I \ominus B$, avec $B$ l'élément structurant.

<img src="https://homepages.inf.ed.ac.uk/rbf/HIPR2/figs/erodbin.gif">

L'érosion recherche une correspondance parfaite entre la région d'image analysée et l'ES : un pixel ne gardera sa valeur de 1 que si ses voisins incluent l'ES (donc valent 1 si le pixel de l'ES qui leur correspond vaut 1 aussi). Vu que les pixels au bord des objets ne vont généralement pas correspondre à l'élément structurant (ayant du noir à proximité), cette opération aura tendance à les faire disparaitre, et donc à ronger les bordures.

##### Avec OpenCV:

```Python
morphed = cv2.erode(img, element_structurant, nombre_de_passages)
```
L'élément structurant doit être une matrice, ou une image chargée en tant que matrice.

In [0]:
pass_slider = widgets.IntSlider(min=1, max=10, step=1, value=1)
es_size_slider = widgets.IntSlider(min=3, max=15, step=2, value=3)

@interact
def erode(image=[f for f in os.listdir(morpho_path) if os.path.isfile(os.path.join(morpho_path, f))], 
          invert=False, n_passages=pass_slider, taille_es=es_size_slider):
    
    # Importation
    img = Image.open(morpho_path + image).convert("L")
    
    if invert:
        img = ImageOps.invert(img)
    
    # Binarisation (0 & 1)
    img = np.array(1.0 * (np.array(img) > 127), dtype="uint8")
    
    # On créé l'élément structurant
    es = np.ones((taille_es,taille_es), dtype=np.int)

    # On morph
    morphed = cv2.erode(img, es, iterations=n_passages)

    # On affiche
    affichage_1x3(img, es, morphed)
    
    display(Markdown("**Consignes**: Observez l'effet du nombre de passes et de la taille du filtre sur le résultat !"))

interactive(children=(Dropdown(description='image', options=('0_M.jpg', '10_text.png', '11_text.jpg', '1_j.png…

### II.1.b Dilatation:

**La dilatation** (*Dilation / Grow / Expand*) est la seconde opération morphologique de base, qui **étends les bordures** des objets / régions saillantes (*foreground regions*), i.e. là où les pixels sont blancs (valent 1). 

Cas d'utilisation de la dilatation :
* Eliminer du bruit noir (*Pepper*) des objets d'intérêt (*foreground*) de l'image.
* Elargir / renforcer les formes d'intérêt.

Mathématiquement, la dilatation est formalisée comme : $I' = I \oplus B$, avec $B$ l'élément structurant.

<img src="https://homepages.inf.ed.ac.uk/rbf/HIPR2/figs/diltbin.gif">

Lors d'une dilatation, un pixel ne gardera sa valeur de 0 que si ses voisins excluent l'ES (donc valent 0 si le pixel de l'ES qui leur correspond vaut 1). Vu que les pixels au bord des objets ne vont généralement pas correspondre à l'élément structurant (a cause du blanc de l'objet à proximité), cette opération aura tendance à rajouter des pixels blancs autour des bordures, et donc à les épaissir.

<u>Remarque</u>: La dilatation est la réciproque de l'érosion: éroder les pixels blancs avec un ES donné revient à dilater les pixels noirs (i.e. *background*) avec le même ES. Si l'on inverse une image N&B, l'érosion aura alors l'effet d'une dilatation, et vice-versa.

##### Avec OpenCV:
```Python
morphed = cv2.dilate(img, element_structurant, iterations)
```

In [0]:
pass_slider = widgets.IntSlider(min=1, max=10, step=1, value=1)
es_size_slider = widgets.IntSlider(min=3, max=15, step=2, value=3)

@interact
def dilate(image=[f for f in os.listdir(morpho_path) if os.path.isfile(os.path.join(morpho_path, f))], 
          invert=False, n_passages=pass_slider, taille_es=es_size_slider):
    
    # Importation
    img = Image.open(morpho_path + image).convert("L")
    
    if invert:
        img = ImageOps.invert(img)
    
    # Binarisation (0 & 1)
    img = np.array(1.0 * (np.array(img) > 127), dtype="uint8")
    
    # On créé l'élément structurant
    es = np.ones((taille_es,taille_es), dtype=np.int)

    # On morph
    morphed = cv2.dilate(img, es, iterations=n_passages)

    # On affiche
    affichage_1x3(img, es, morphed)
    
    display(Markdown("**Consignes**: Observez l'effet du nombre de passes et de la taille du filtre sur le résultat !"))

interactive(children=(Dropdown(description='image', options=('fish30.png', 'skyhawk.png', 'handbent1.png', 'ph…

### <span style="color:crimson">**[<u>Exercice</u>]** A vous de jouer:</span>
***
<div style="color:DarkSlateBlue">  
 
1. **Ajouter du bruit S&P (via un slider interactif) à l'image sélectionnée, et observer les effets de l'érosion / dilatation dessus.**
    
    
2. **En employant uniquement une érosion (ou une dilatation), proposez une solution permettant d'extraire les bordures des objets d'une image binaire.**
    
    
3. **Implémentez le <u>gradient morphologique</u> de l'image: différence entre la dilatation et l'érosion d'une image.**
    
</div>

In [0]:
# > Emplacement exercice <



## <span style="color: DodgerBlue;text-decoration: underline">II.2 Ouverture et Fermeture</span>

In [0]:
### Code utile pour cette section :
import os

from PIL import Image, ImageOps
from skimage.util import random_noise
import numpy as np
import cv2

import seaborn as sns
from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt
%matplotlib inline

import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import display, Markdown

def affichage_1x3(img1, img2, img3, grid=True):
    fig = plt.figure(figsize=(18, 6))
    gs  = GridSpec(1, 3, width_ratios=[1, 0.4, 1])
    display_axes = "both" if grid else "none"
    
    plt.subplot(gs[0]), plt.imshow(img1, cmap='gray'), plt.title("Image", color='b')
    
    plt.subplot(gs[1]), plt.imshow(img2, cmap='gray', extent=(0, img2.shape[0], img2.shape[1], 0), origin='upper')
    plt.title("ES"), plt.locator_params(nbins=len(img2)), plt.grid(axis=display_axes, color='b', linewidth=1)
    
    plt.subplot(gs[2]), plt.imshow(img3, cmap='gray'), plt.title("Resultat", color='b')
    plt.show()

### II.2.a Ouverture:

**Une ouverture** (*Opening*) est l'enchaînement d'une érosion et d'une dilatation : $I' = opening(I) = dilation(erosion(I))$

Mathématiquement, l'ouverture est formalisée comme : $I' = I \circ B = (I\ominus B)\oplus B$, avec $B$ l'élément structurant.

<img src="https://opencv-python-tutroals.readthedocs.io/en/latest/_images/opening.png">    
    
<img src="http://what-when-how.com/wp-content/uploads/2012/07/tmp26dc191_thumb2.png">

C'est une opération très utilisée en restoration d'image pour traiter le bruit *Salt* dans l'arrière-plan : en effet, l'erosion  va permettre d'éliminer le bruit de fond non voulu, mais va également ronger les bords des objets eux-mêmes. La dilatation successive va donc permettre de restaurer les bords à (presque) leur valeur d'origine, mais sans restaurer le bruit.

##### Avec OpenCV:
```Python
morphed = cv2.morphologyEx(img, cv2.MORPH_OPEN, es, iterations)
```

In [0]:
pass_slider = widgets.IntSlider(min=1, max=10, step=1, value=1)
es_size_slider = widgets.IntSlider(min=3, max=15, step=2, value=3)

@interact
def closing(image=[f for f in os.listdir(morpho_path) if os.path.isfile(os.path.join(morpho_path, f))], 
          invert=False, n_passages=pass_slider, taille_es=es_size_slider):
    
    # Importation
    img = Image.open(morpho_path + image).convert("L")
    
    if invert:
        img = ImageOps.invert(img)
    
    # Binarisation (0 & 1)
    img = np.array(1.0 * (np.array(img) > 127), dtype="uint8")
    
    # On créé l'élément structurant
    es = np.ones((taille_es,taille_es), dtype=np.int)

    # On morph
    morphed = cv2.morphologyEx(img, cv2.MORPH_OPEN, es, iterations=n_passages)

    # On affiche
    affichage_1x3(img, es, morphed)
    
    display(Markdown("**Consignes**: Observez l'effet du nombre de passes et de la taille du filtre sur le résultat !"))

interactive(children=(Dropdown(description='image', options=('fish30.png', 'skyhawk.png', 'handbent1.png', 'ph…

### II.2.b Fermeture:

**Une fermeture** (*Closing*) est l'enchaînement d'une dilatation et d'une érosion : $I' = closing(I) = erosion(dilation(I))$


Mathématiquement, la fermeture est formalisée comme : $I' = I \bullet B = (I\oplus B)\ominus B$, avec $B$ l'élément structurant.

<img src="https://opencv-python-tutroals.readthedocs.io/en/latest/_images/closing.png">
<img src="http://what-when-how.com/wp-content/uploads/2012/07/tmp26dc188_thumb2.png">

C'est une opération très utilisée en restoration d'image pour traiter le bruit *Pepper* dans les objets : en effet, la dilatation va permettre de remplir les "trous" non voulus dans un objet (*pepper noise*), mais va également étendre les bords des objets eux-mêmes. L'érosion successive va donc permettre de restaurer les bords à (presque) leur valeur d'origine, mais sans restaurer le bruit.

<u>Remarque</u>: La fermeture est la réciproque de l'ouverture: ouvrir les pixels blancs avec un ES donné revient à fermer les pixels noirs avec le même ES. Si l'on inverse une image N&B, l'ouverture aura alors l'effet d'une fermeture, et vice-versa.

##### Avec OpenCV:
```Python
morphed = cv2.morphologyEx(img, cv2.MORPH_CLOSE, es, iterations)
```

In [0]:
pass_slider = widgets.IntSlider(min=1, max=10, step=1, value=1)
es_size_slider = widgets.IntSlider(min=3, max=15, step=2, value=3)

@interact
def closing(image=[f for f in os.listdir(morpho_path) if os.path.isfile(os.path.join(morpho_path, f))], 
          invert=False, n_passages=pass_slider, taille_es=es_size_slider):
    
    # Importation
    img = Image.open(morpho_path + image).convert("L")
    
    if invert:
        img = ImageOps.invert(img)
    
    # Binarisation (0 & 1)
    img = np.array(1.0 * (np.array(img) > 127), dtype="uint8")
    
    # On créé l'élément structurant
    es = np.ones((taille_es,taille_es), dtype=np.int)

    # On morph
    morphed = cv2.morphologyEx(img, cv2.MORPH_CLOSE, es, iterations=n_passages)

    # On affiche
    affichage_1x3(img, es, morphed)
    
    display(Markdown("**Consignes**: Observez l'effet du nombre de passes et de la taille du kernel sur le résultat !"))

interactive(children=(Dropdown(description='image', options=('fish30.png', 'skyhawk.png', 'handbent1.png', 'ph…

### <span style="color:crimson">**[<u>Exercice</u>]** A vous de jouer:</span>
***
<div style="color:DarkSlateBlue">  
 
1. **Créez une méthode interactive qui permet:**
    * D'ajouter du bruit *Salt*, *Pepper* ou *Salt&Pepper*
    * D'afficher dans une grille 3x2 : ([Image d'origine, Image bruitée], [Erodée, Dilatée], [Ouverte, Fermée])
    
Essayez votre fonction sur les différentes images de démonstration et observez les différences d'effets de chaque opération.

2. **Implémentez une chaine de traitement qui va :**
    1. Ajouter du bruit S&P à une image
    2. Le nettoyer
    3. Extraire les bordures des objets de cette image
    4. (Si nécessaire) Nettoyer les portions/pixels non-utiles de l'image *edge* extraites


3. **Implémentez les deux opérations suivantes et observez:**
    1. **Top-Hat**: image - ouverture(image). 
        Cette opération isole les zones plus petites et de forme similaire à l'élément structurant et plus claires que leur entourage.
    2. **Black-Hat**: fermeture(image) - image. 
        Cette opération isole les zones plus petites et de forme similaire à l'élément l'élément structurant et plus sombres que leur entourage.

> <u>Astuce</u>: vous pouvez utiliser les méthodes spécifiques d'OpenCV pour comparer vos résultats :
```Python
morph = cv2.morphologyEx(img, cv2.MORPH_TOPHAT, es)
morph = cv2.morphologyEx(img, cv2.MORPH_BLACKHAT, es)
```
    
4. **En partant de l'image N&B des cellules (`9_cells.png`), essayez d'en extraire les contours les plus propres possibles (en utilisant des opérations morphologiques, pre ou post extraction de contours). Réappliquez les contours obtenus à l'image d'origine de sorte à les faire ressortir (amplifier leur luminosité).**
    
</div>

In [0]:
# > Emplacement exercice <



## <span style="color: DodgerBlue;text-decoration: underline">II.3 Hit-or-Miss</span>

*The **Hit-or-miss transform** is an operation that detects a given configuration (or pattern) in a binary image, using the morphological erosion operator and a pair of disjoint structuring elements. The result of the hit-or-miss transform is the set of positions where the first structuring element fits in the foreground of the input image, and the second structuring element misses it completely.*

<img src="https://homepages.inf.ed.ac.uk/rbf/HIPR2/figs/hamcrn.gif">

*Here, we use two structuring elements (say B1 and B2). We ask a simple question : does B1 fits the object while, simultaneously, B2 misses the object (i.e. fits the background) ? In other words, we are interested only in those pixels whose neighborhood exactly matches B1 while not matching B2 at the same time.*

Mathématiquement, la transformée Hit-or-Miss est formalisée comme : $I' = I\odot B=(I\ominus B_1)\cap (I^c\ominus B_2)$

Avec :
* $B_1$ et $B_2$ sont éléments structurants **disjoints**, dont la combinaison donne $B$.
* $I^c$ le complément de $I$ : ($I^c = 1 - I$)

<img src="https://i1.wp.com/theailearner.com/wp-content/uploads/2019/07/hitmiss.png?w=759&ssl=1">

Contrairement aux opérations morphologiques précédentes, les *kernels* des H|M n'est plus binaire, mais trinaire:
* 1 (blanc): il faut qu'il y ait 1 dans ce pixel
* 0 (gris): on s'en fiche de ce qu'il y à dans ce pixel
* -1 (noir): il faut qu'il y ait 0 dans ce pixel

L'application de l'ES illustré donnerait :

<img src="https://i2.wp.com/theailearner.com/wp-content/uploads/2019/07/hitormiss-1.png?w=748&ssl=1">

La transformation hit-or-miss est utilisée pour détecter des patterns spécifiques dans une image. Elle peut être vue comme une détection de **gradient (2D) orienté.**

*The hit-or-miss transform has many applications in more complex morphological operations: it is being used to construct the thinning, thickening and convex hull operators*

In [0]:
### Code utile pour cette section :
import os

from PIL import Image, ImageOps
from skimage.util import random_noise
import numpy as np
import cv2

import seaborn as sns
from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt
%matplotlib inline

import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import display, Markdown

def affichage_1x3(img1, img2, img3, grid=True):
    fig = plt.figure(figsize=(18, 6))
    gs  = GridSpec(1, 3, width_ratios=[1, 0.4, 1])
    display_axes = "both" if grid else "none"
    
    plt.subplot(gs[0]), plt.imshow(img1, cmap='gray'), plt.title("Image", color='b')
    
    plt.subplot(gs[1]), plt.imshow(img2, cmap='gray', extent=(0, img2.shape[0], img2.shape[1], 0), origin='upper')
    plt.title("ES"), plt.locator_params(nbins=len(img2)), plt.grid(axis=display_axes, color='b', linewidth=1)
    
    plt.subplot(gs[2]), plt.imshow(img3, cmap='gray'), plt.title("Resultat", color='b')
    plt.show()

##### Avec OpenCV:
```Python
morphed = cv2.morphologyEx(img, cv2.MORPH_HITMISS, es, iterations)
```

In [0]:
pass_slider = widgets.IntSlider(min=1, max=10, step=1, value=1)
resize_slider = widgets.FloatSlider(min=0.2, max=5.0, step=0.2, value=1)

@interact
def hit_and_miss(image=[f for f in os.listdir(morpho_path) if os.path.isfile(os.path.join(morpho_path, f))], 
          invert=False, n_passages=pass_slider, size=resize_slider):
    
    # Importation
    img = Image.open(morpho_path + image).convert("L")
    
    if invert:
        img = ImageOps.invert(img)
    
    # Binarisation (0 & 1)
    img = np.array(1.0 * (np.array(img) > 127), dtype="uint8")
    
    # On créé l'élément structurant
    es = np.array([[-1,-1,-1],[1,1,1],[0,0,0]], dtype="int")
    es = cv2.resize(es, None, fx=size, fy=size, interpolation=cv2.INTER_NEAREST)
    
    # On morph
    morphed = cv2.morphologyEx(img, cv2.MORPH_HITMISS, es, iterations=n_passages)

    # On affiche
    affichage_1x3(img, es, morphed)

interactive(children=(Dropdown(description='image', options=('fish30.png', 'skyhawk.png', 'handbent1.png', 'ph…

## <span style="color: DodgerBlue;text-decoration: underline">II.4 Thinning & Thickening</span>

In [0]:
### Code utile pour cette section :
import os

from PIL import Image, ImageOps
from skimage.util import random_noise
import skimage.morphology as skimorph
import numpy as np
import cv2

import seaborn as sns
from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt
%matplotlib inline

import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import display, Markdown

def affichage_1x3(img1, img2, img3, grid=True):
    fig = plt.figure(figsize=(18, 6))
    gs  = GridSpec(1, 3, width_ratios=[1, 0.4, 1])
    display_axes = "both" if grid else "none"
    
    plt.subplot(gs[0]), plt.imshow(img1, cmap='gray'), plt.title("Image", color='b')
    
    plt.subplot(gs[1]), plt.imshow(img2, cmap='gray', extent=(0, img2.shape[0], img2.shape[1], 0), origin='upper')
    plt.title("ES"), plt.locator_params(nbins=len(img2)), plt.grid(axis=display_axes, color='b', linewidth=1)
    
    plt.subplot(gs[2]), plt.imshow(img3, cmap='gray'), plt.title("Resultat", color='b')
    plt.show()
    
def affichage_1x2(img1,img2):
    fig = plt.figure(figsize=(12, 6))
    
    plt.subplot(121), plt.imshow(img1, cmap='gray'), plt.title("Image", color='b')
    plt.subplot(122), plt.imshow(img2, cmap='gray'), plt.title("Resultat", color='b')
    plt.show()

### II.4.a Thinning:

*__Thinning__ is used to thin the foreground region such that its extent and connectivity are preserved, producing a **topologically equivalent** image. Preserving extent means preserving the endpoints of a structure, whereas preserving connectivity can refer to either 4-connected or 8-connected lines.*

*Pixel connectivity illustration:*

<img src="https://images.deepai.org/converted-papers/1906.03366/images/Connectedness.jpg" width=300>

*Thinning is mostly used for producing skeletons (which serve as image descriptors), and for reducing the output of the edge detectors to a one-pixel thickness.*

<img src="https://i0.wp.com/theailearner.com/wp-content/uploads/2019/07/thinresult.png?resize=1024%2C141">

Thinning can be formalized based on the Hit-or-Miss transform : $I\otimes B = I - (I\odot B)$

*The choice of structuring element determines under what situations a foreground pixel will be set to background, and hence it determines the application for the thinning operation. The binary structuring elements used for thinning are of the extended type described under the hit-and-miss transform (i.e. they can contain both ones and zeros).*

*The operator is normally applied repeatedly until it causes no further changes to the image (i.e. until convergence). Alternatively, in some applications, e.g. pruning, the operations may only be applied for a limited number of iterations.*

#### Exemple via Skimage (module `morphology`) :

In [0]:
@interact
def thinning(image=[f for f in os.listdir(morpho_path) if os.path.isfile(os.path.join(morpho_path, f))], invert=False):
    
    # Importation
    img = Image.open(morpho_path + image).convert("L")
    
    if invert:
        img = ImageOps.invert(img)
    
    # Binarisation (0 & 1)
    img = np.array(1.0 * (np.array(img) > 127), dtype="uint8")

    # On morph
    morphed = skimorph.thin(img)

    # On affiche
    affichage_1x2(img, morphed)

interactive(children=(Dropdown(description='image', options=('fish30.png', 'skyhawk.png', 'handbent1.png', 'ph…

#### Exemple en combinant érosions et dilatations (via OpenCV) sur une image générée :

In [0]:
def create_text_image(size, text, thick=5):
    img = np.zeros(size, dtype='uint8')
    cv2.putText(img, text, org=(int(0.3*size[0]), int(0.3*size[1])), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=3, color=(255), thickness=thick, lineType=cv2.LINE_AA)
    return img

@interact
def thinning2(thick=(2,20,2)):
    
    # Créons une image de démo
    img = create_text_image((200,400), "IBIOM", thick)
    img1 = img.copy()
    
    # Binarisation (0 & 1)
    #img = np.array(1.0 * (np.array(img) > 127), dtype="uint8")
    
    # On créé l'élément structurant
    es = cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3))
    
    # Create an empty output image to hold values
    morphed = np.zeros(img.shape, dtype='uint8')

    # On morph
    # Loop until erosion leads to an empty set
    while (cv2.countNonZero(img1)!=0):
        # Erosion
        erode = cv2.erode(img1, es)
        # Opening on eroded image
        opening = cv2.morphologyEx(erode, cv2.MORPH_OPEN, es)
        # Subtract these two
        subset = erode - opening
        # Union of all previous sets
        morphed = cv2.bitwise_or(subset, morphed)
        # Set the eroded image for next iteration
        img1 = erode.copy()

    # On affiche
    affichage_1x3(img, es, morphed)

interactive(children=(IntSlider(value=10, description='thick', max=20, min=2, step=2), Output()), _dom_classes…

### II.4.b Thickening:

*__Thickening__ is a morphological operation that is used to grow selected regions of foreground pixels in binary images, somewhat like dilation or closing. It has several applications such as determining the approximate convex hull of a shape.*

*The thickened image consists of the original image plus any additional foreground pixels switched on by the hit-and-miss transform.*

*Thickening is the dual of thinning, i.e. thickening the foreground is equivalent to thinning the background.*

### <span style="color:crimson">**[<u>Exercice</u>]** A vous de jouer:</span>
***
<div style="color:DarkSlateBlue">  

1. **Générez de nouveaux ES comprenant des zones *hit* et/ou *miss*:**
    1. Créez différents types d'éléments structurants (via numpy, ou via `cv2getStructuringElement()`) 
    2. Les sauvegarder sous forme d'images binaires dans le dossier `img/es` de votre TP pour pouvoir les réutiliser.
    3. Les appliquer aux différentes images du TP et observez.
    
> <u>Astuces</u>:
```python
cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
cv2.getStructuringElement(cv2.MORPH_CROSS, (5,5))
```
> Vous pouvez également vous servir de `cv2.resize(es, None, fx=rate, fy=rate, interpolation=cv2.INTER_NEAREST)` pour redimensionner des ES existants.
    
> Vous pouvez vous servir des méthodes suivantes pour convertir un ES du format image (0-255) au binaire (0-1):
```python
def i2b(f):
    return np.where(f==0, 1, 0)
def b2i(f):
    return np.where(f==1, 0, 255)
```
    
    
2. **Appliquez la méthode de Thinning d'OpenCV aux différentes images du TP (`img/morpho/`) et observez.**

    
2. **Créez une image carrée à fond noir sur laquelle vous tracerez une grille de 5x5 lignes blanches de 1 pixel de large, équidistantes entre elles. Trouvez l'ES permettant d'extraire les intersections des traits horizontaux et verticaux de cette image.**
    
> <u>Astuce</u>: Pour dessiner une ligne
```python
cv2.line(img, pt1, pt2, color)
```
    
    
3. **[<u>Bonus</u>] A partir des intersections extraites précédemment, récoltez les coordonnées (x,y) des pixels blancs et tracez des lignes entre les points alignés verticalement et horizontalement, de sorte à essayer de re-créer l'image d'origine.**

    
4. **Reprenez votre programme du `II.2` et tentez d'améliorer l'extraction de contours grace au Thinning**

</div>

In [0]:
# > Emplacement exercice <



## <span style="color: DodgerBlue;text-decoration: underline">II.5 Skeletonization</span>

*__Skeletonization__ (or Medial axis transform) is a process for reducing foreground regions in a binary image to a skeletal remnant that largely preserves the extent and connectivity of the original region while throwing away most of the original foreground pixels.*

<img src="http://zone.ni.com/images/reference/en-XX/help/372916T-01/binary_lskeleton01.gif">
<img src="https://scikit-image.org/docs/dev/_images/sphx_glr_plot_morphology_008.png">

*The skeleton can be produced in two main ways :*
* _The first is to use some kind of **morphological thinning** that successively erodes away pixels from the boundary (while preserving the end points of line segments) until no more thinning is possible, at which point what is left approximates the skeleton._ 
* _The alternative method is to first calculate the **distance transform** of the image. The skeleton then lies along the singularities (i.e. creases or curvature discontinuities) in the distance transform._

In [0]:
### Code utile pour cette section :
import os

from PIL import Image, ImageOps
from skimage.util import random_noise
import skimage.morphology as skimorph
import numpy as np
import cv2

import seaborn as sns
from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt
%matplotlib inline

import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import display, Markdown

def affichage_1x3(img1, img2, img3, grid=True):
    fig = plt.figure(figsize=(18, 6))
    gs  = GridSpec(1, 3, width_ratios=[1, 0.4, 1])
    display_axes = "both" if grid else "none"
    
    plt.subplot(gs[0]), plt.imshow(img1, cmap='gray'), plt.title("Image", color='b')
    
    plt.subplot(gs[1]), plt.imshow(img2, cmap='gray', extent=(0, img2.shape[0], img2.shape[1], 0), origin='upper')
    plt.title("ES"), plt.locator_params(nbins=len(img2)), plt.grid(axis=display_axes, color='b', linewidth=1)
    
    plt.subplot(gs[2]), plt.imshow(img3, cmap='gray'), plt.title("Resultat", color='b')
    plt.show()
    
def affichage_1x2(img1,img2):
    fig = plt.figure(figsize=(12, 6))
    
    plt.subplot(121), plt.imshow(img1, cmap='gray'), plt.title("Image", color='b')
    plt.subplot(122), plt.imshow(img2, cmap='gray'), plt.title("Resultat", color='b')
    plt.show()

#### Exemple via Skimage (module `morphology`) :

In [0]:
@interact
def skeletonize(image=[f for f in os.listdir(morpho_path) if os.path.isfile(os.path.join(morpho_path, f))], invert=False):
    
    # Importation
    img = Image.open(morpho_path + image).convert("L")
    
    if invert:
        img = ImageOps.invert(img)
    
    # Binarisation (0 & 1)
    img = np.array(1.0 * (np.array(img) > 127), dtype="uint8")

    # On morph
    morphed = skimorph.skeletonize(img == 1)

    # On affiche
    affichage_1x2(img, morphed)
    
    display(Markdown("**Consignes**: Observez les effets sur l'image de cellules, et sur leur inverse !"))

interactive(children=(Dropdown(description='image', options=('0_M.jpg', '10_text.png', '11_text.jpg', '1_j.png…

## <span style="color: DodgerBlue;text-decoration: underline">II.6 Convex Hull</span>

*The **convex hull** is the set of pixels included in the smallest convex polygon that surround all white pixels in the input image.*

<img src="https://scikit-image.org/docs/dev/_images/sphx_glr_plot_morphology_009.png">

#### Exemple via Skimage (module `morphology`) :

In [0]:
@interact
def convex_hull(image=[f for f in os.listdir(morpho_path) if os.path.isfile(os.path.join(morpho_path, f))], invert=False):
    
    # Importation
    img = Image.open(morpho_path + image).convert("L")
    
    if invert:
        img = ImageOps.invert(img)
    
    # Binarisation (0 & 1)
    img = np.array(1.0 * (np.array(img) > 127), dtype="uint8")

    # On morph
    morphed = skimorph.convex_hull_image(img == 1)

    # On affiche
    affichage_1x2(img, morphed)

interactive(children=(Dropdown(description='image', options=('fish30.png', 'skyhawk.png', 'handbent1.png', 'ph…

### <span style="color:crimson">**[<u>Exercice</u>]** A vous de jouer:</span>
***
<div style="color:DarkSlateBlue">  
    
**Votre objectif lors de cet exercice sera de combiner les différents outils à votre disposition (manipulation de luminance, débruitage, opération morphologiques) afin de segmenter l'image des cellules le plus proprement possible.**
    
> <u>Astuce</u>: Vous pouvez soustraire / ajouter, ou combiner (and/or) les masques entre eux (ou encore les masques avec l'image d'origine) pour obtenir de nouvelles images plus propices à la segmentation, ou a l'application d'autres traitements.
    
**Pour cela, vous allez devoir :**

1. **Pré-traiter et binariser l'image :**
    * Inverser l'image (les cellules sont suposées être l'information à détecter)
    * Modifiez la répartition de luminance / contraste de l'image de sorte à faciliter la binarisation (min-max, equ, log ...). Vous pouvez également manipuler les seuils de ces opérations pour obtenir un résultat favorable.
    * Déterminer le seuil de binarisation/segmentation optimal pour l'objectif de l'exercice.
    
    
2. **Nettoyage du masque obtenu afin de retirer le bruit du background (éléments n'étant pas des cellules)**
    
> <u>Astuce</u>: vous pouvez vous servir :
* De `skimage.morphology.remove_small_objects` pour retirer les petits objets de l'image
* Ou encore `skimage.morphology.binary_closing(img, disk(3))` qui permet de spécifier les ES plus aisément.
    
    
3. **Superposez le masque final avec l'image d'origine afin de segmenter les cellules.**
    

4. **[<u>Bonus</u>] Servez-vous du masque final pour compter le nombre de cellules présentes sur l'image.**
    
> <u>Astuces</u>:
* Vous pouvez vous servir de la méthode `cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)` sur le masque, contours que vous pouvez ensuite dessiner avec `cv2.drawContours`
* Ou encore de `skimage.morphology.label` pour regrouper et labeliser automatiquement les blobs (groupes de pixels connectés / de luminance similaire).

</div>

In [0]:
# > Emplacement exercice <



<div style="color:Navy"> 

***
# Fin du TP3
***
    
</div>